In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import EarlyStopping
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

### Pipeline psuedo code
    n = look back window
    k = number of PCs to keep

    for each time point t:
        p = number of stocks in investable universe at time t
        Define an n x p feature matrix X (lagged returns)

        Perform PCA on X
        Keep the first k PCs in an n x k matrix Z

    for each stock s in the investable universe at time t:
        Define an n x 1 outcome vector y (future returns of stock s)
        Perform a linear regression of y on Z
        Predict y for stock s at time t+1

## Data Preparation

In [21]:
returns = pd.read_pickle("./Data/returns.pkl")
returns = returns.iloc[1:]

In [22]:
drop_columns = []

for col in returns.columns:
    if returns[col].isnull().all() == True:
        drop_columns.append(col)
        
returns.drop(columns=drop_columns, inplace=True)

## Pipeline

In [23]:
def get_investable(t, n):
    """"Find stocks in investable universe at time t
    (stocks in the S&P500 that have prices recorded for the last n days)"""
    
    df_investable = returns.copy(deep = True).sort_index(ascending = False)
    
    #add 1 date to get the test features in investable
    t = t + pd.DateOffset(1)
    n += 1
    
    #if t is now a non-trading day, advance until we reach a valid trading day
    while t not in df_investable.index:
        t = t + pd.DateOffset(1)
    
    t_index = df_investable.index.get_loc(t)
    
    #take n_rows worth of data upto time specified
    df_investable = df_investable.iloc[t_index + 1:t_index + n + 1]
    
    #find all stocks that exist in the S&P at this time period
    investable_universe = []
    for col in df_investable.columns:
        if ~df_investable[col].iloc[:n].isna().any():
            investable_universe.append(col)
        
    df_investable = df_investable[investable_universe]
    
    return df_investable

In [24]:
def apply_PCA(inv, k):
    pca = PCA(n_components = k) 
    inv_scaled = StandardScaler().fit_transform(inv)   
    principal_components = pca.fit_transform(inv_scaled)

    df = pd.DataFrame(data = principal_components)
    
    #For explained variance table
    components = pca.components_
    component_explained_var = pca.explained_variance_ratio_ * 100
    
    comp_names = ['PCA' + str(i) for i in range(1, len(component_explained_var) + 1)]

    pca_results = pd.DataFrame(data = component_explained_var, index = comp_names)
    pca_results.columns = ['Explained variance (%)']
    pca_results['Explained variance (%)'] = pca_results['Explained variance (%)'].round(2)
    
    return df

In [25]:
def define_y(inv, stock, n):
    y = inv[[stock]].iloc[:n+1]
    
    return y

In [26]:
def train_test(X, y):
    X_train = X.iloc[1:, :]
    X_test = X.iloc[0:1, :]
    y_train = y.iloc[1:]
    y_test = y.iloc[0:1]
    
    return X_train, y_train, X_test, y_test

In [48]:
def reshape(X_train, y_train, X_test, y_test, timesteps):
    X_train = X_train.values.reshape(-1, timesteps, X_train.shape[1])
    X_test = X_test.values.reshape(-1, 1, X_test.shape[1])
    y_train = y_train.values
    y_test = y_test.values
    
    return X_train, y_train, X_test, y_test

In [ ]:
def model_fit(X_train, y_train):
    model = Sequential()
    model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(y_train.shape[0]))
    
    model.compile(loss = 'mse', optimizer = 'adam')
    
    early_stopping = EarlyStopping(monitor="val_loss", min_delta=1e-5, patience=20)
    
    history = model.fit(X_train, y_train, epochs=300, batch_size=16,
                        verbose = 0, validation_split = 0.2, 
                       callbacks = [early_stopping])
    
    return model

In [29]:
def model_predict(model, X_test):
    yhat = model.predict(X_test, verbose = 0)
    
    return yhat

In [55]:
def predict_returns(t, n, k, timesteps):
    inv = get_investable(t, n)
    X = apply_PCA(inv, k)
    
    returns_t = pd.DataFrame(index = inv.columns, columns = ['Pred', 'Actual'])
    
    for stock in inv.columns:
        y = define_y(inv, stock, n)
        X_train, y_train, X_test, y_test = train_test(X, y)
        X_train, y_train, X_test, y_test = reshape(X_train, y_train, X_test, y_test, timesteps)
        model = model_fit(X_train, y_train)
        yhat = model_predict(model, X_test)[0][0]
        returns_t['Pred'].loc[stock] = yhat*100
        returns_t['Actual'].loc[stock] = y_test
    
    return returns_t

In [31]:
def rank_stocks(returns, num_stocks):
    pred_returns = returns.sort_values(by = 'Pred', ascending = False)
    topn = pred_returns.head(num_stocks)
    botn = pred_returns.tail(num_stocks)
    
    return topn, botn

In [32]:
def portfolio_return(topn, botn, returns):
    return_t = topn['Actual'].mean() - botn['Actual'].mean()
    
    return return_t

In [53]:
def pipeline(n, k, timesteps, num_stocks):

    time_range = returns.loc['2007':'2021'].index
    
    portfolio = pd.DataFrame(index = time_range, columns = ['Portfolio Return'])
    
    count = 0
    for t in time_range[0:1]:
        pred_actual = predict_returns(t, n, k, timesteps)
        topn, botn = rank_stocks(pred_actual, num_stocks)
        return_t = portfolio_return(topn, botn, pred_actual)
        portfolio['Portfolio Return'].loc[t] = return_t
        count +=1
        print(f'{(count/len(time_range))*100:.2f}% complete')
    
    return portfolio

In [56]:
portfolio = pipeline(200, 20, 50, 5)

Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 0.9884 - val_loss: 1.1047
Epoch 2/100
1/1 [==============================] - 0s 51ms/step - loss: 0.9693 - val_loss: 1.1011
Epoch 3/100
1/1 [==============================] - 0s 64ms/step - loss: 0.9518 - val_loss: 1.0972
Epoch 4/100
1/1 [==============================] - 0s 40ms/step - loss: 0.9351 - val_loss: 1.0930
Epoch 5/100
1/1 [==============================] - 0s 42ms/step - loss: 0.9190 - val_loss: 1.0886
Epoch 6/100
1/1 [==============================] - 0s 46ms/step - loss: 0.9030 - val_loss: 1.0841
Epoch 7/100
1/1 [==============================] - 0s 46ms/step - loss: 0.8870 - val_loss: 1.0794
Epoch 8/100
1/1 [==============================] - 0s 42ms/step - loss: 0.8708 - val_loss: 1.0746
Epoch 9/100
1/1 [==============================] - 0s 43ms/step - loss: 0.8543 - val_loss: 1.0696
Epoch 10/100
1/1 [==============================] - 0s 48ms/step - loss: 0.8371 - val_loss: 1.0645
Epoch 11/100
1/1 [===

1/1 [==============================] - 0s 53ms/step - loss: 1.4775 - val_loss: 0.6945
Epoch 5/100
1/1 [==============================] - 0s 50ms/step - loss: 1.4524 - val_loss: 0.6960
Epoch 6/100
1/1 [==============================] - 0s 51ms/step - loss: 1.4271 - val_loss: 0.6971
Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 1.6640 - val_loss: 0.2218
Epoch 2/100
1/1 [==============================] - 0s 61ms/step - loss: 1.6340 - val_loss: 0.2190
Epoch 3/100
1/1 [==============================] - 0s 55ms/step - loss: 1.6055 - val_loss: 0.2162
Epoch 4/100
1/1 [==============================] - 0s 52ms/step - loss: 1.5779 - val_loss: 0.2133
Epoch 5/100
1/1 [==============================] - 0s 48ms/step - loss: 1.5507 - val_loss: 0.2105
Epoch 6/100
1/1 [==============================] - 0s 51ms/step - loss: 1.5235 - val_loss: 0.2078
Epoch 7/100
1/1 [==============================] - 0s 50ms/step - loss: 1.4957 - val_loss: 0.2051
Epoch 8/100
1/1 [=================

Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 0.1575 - val_loss: 0.7442
Epoch 2/100
1/1 [==============================] - 0s 66ms/step - loss: 0.1464 - val_loss: 0.7482
Epoch 3/100
1/1 [==============================] - 0s 59ms/step - loss: 0.1363 - val_loss: 0.7520
Epoch 4/100
1/1 [==============================] - 0s 52ms/step - loss: 0.1272 - val_loss: 0.7557
Epoch 5/100
1/1 [==============================] - 0s 57ms/step - loss: 0.1188 - val_loss: 0.7592
Epoch 6/100
1/1 [==============================] - 0s 53ms/step - loss: 0.1112 - val_loss: 0.7625
Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 0.5431 - val_loss: 0.2958
Epoch 2/100
1/1 [==============================] - 0s 63ms/step - loss: 0.5264 - val_loss: 0.2998
Epoch 3/100
1/1 [==============================] - 0s 50ms/step - loss: 0.5104 - val_loss: 0.3040
Epoch 4/100
1/1 [==============================] - 0s 51ms/step - loss: 0.4949 - val_loss: 0.3083
Epoch 5/100
1/1 [=======

1/1 [==============================] - 0s 50ms/step - loss: 0.4407 - val_loss: 2.0636
Epoch 10/100
1/1 [==============================] - 0s 48ms/step - loss: 0.4238 - val_loss: 2.0602
Epoch 11/100
1/1 [==============================] - 0s 47ms/step - loss: 0.4066 - val_loss: 2.0564
Epoch 12/100
1/1 [==============================] - 0s 47ms/step - loss: 0.3889 - val_loss: 2.0523
Epoch 13/100
1/1 [==============================] - 0s 49ms/step - loss: 0.3707 - val_loss: 2.0478
Epoch 14/100
1/1 [==============================] - 0s 50ms/step - loss: 0.3522 - val_loss: 2.0431
Epoch 15/100
1/1 [==============================] - 0s 50ms/step - loss: 0.3333 - val_loss: 2.0381
Epoch 16/100
1/1 [==============================] - 0s 49ms/step - loss: 0.3142 - val_loss: 2.0329
Epoch 17/100
1/1 [==============================] - 0s 49ms/step - loss: 0.2949 - val_loss: 2.0274
Epoch 18/100
1/1 [==============================] - 0s 51ms/step - loss: 0.2756 - val_loss: 2.0219
Epoch 19/100
1/1 [=====

Epoch 22/100
1/1 [==============================] - 0s 46ms/step - loss: 0.0701 - val_loss: 1.6905
Epoch 23/100
1/1 [==============================] - 0s 44ms/step - loss: 0.0638 - val_loss: 1.6864
Epoch 24/100
1/1 [==============================] - 0s 46ms/step - loss: 0.0579 - val_loss: 1.6824
Epoch 25/100
1/1 [==============================] - 0s 46ms/step - loss: 0.0524 - val_loss: 1.6783
Epoch 26/100
1/1 [==============================] - 0s 45ms/step - loss: 0.0474 - val_loss: 1.6741
Epoch 27/100
1/1 [==============================] - 0s 47ms/step - loss: 0.0428 - val_loss: 1.6701
Epoch 28/100
1/1 [==============================] - 0s 47ms/step - loss: 0.0386 - val_loss: 1.6661
Epoch 29/100
1/1 [==============================] - 0s 46ms/step - loss: 0.0347 - val_loss: 1.6622
Epoch 30/100
1/1 [==============================] - 0s 45ms/step - loss: 0.0312 - val_loss: 1.6585
Epoch 31/100
1/1 [==============================] - 0s 46ms/step - loss: 0.0280 - val_loss: 1.6551
Epoch 32/1

...

1/1 [==============================] - 0s 35ms/step - loss: 0.0057 - val_loss: 0.5777
Epoch 44/100
1/1 [==============================] - 0s 35ms/step - loss: 0.0051 - val_loss: 0.5767
Epoch 45/100
1/1 [==============================] - 0s 38ms/step - loss: 0.0045 - val_loss: 0.5757
Epoch 46/100
1/1 [==============================] - 0s 35ms/step - loss: 0.0041 - val_loss: 0.5748
Epoch 47/100
1/1 [==============================] - 0s 35ms/step - loss: 0.0037 - val_loss: 0.5738
Epoch 48/100
1/1 [==============================] - 0s 36ms/step - loss: 0.0034 - val_loss: 0.5729
Epoch 49/100
1/1 [==============================] - 0s 37ms/step - loss: 0.0032 - val_loss: 0.5720
Epoch 50/100
1/1 [==============================] - 0s 38ms/step - loss: 0.0030 - val_loss: 0.5711
Epoch 51/100
1/1 [==============================] - 0s 40ms/step - loss: 0.0028 - val_loss: 0.5702
Epoch 52/100
1/1 [==============================] - 0s 36ms/step - loss: 0.0026 - val_loss: 0.5693
Epoch 53/100
1/1 [=====

1/1 [==============================] - 0s 45ms/step - loss: 0.2217 - val_loss: 1.1111
Epoch 13/100
1/1 [==============================] - 0s 51ms/step - loss: 0.2110 - val_loss: 1.1053
Epoch 14/100
1/1 [==============================] - 0s 43ms/step - loss: 0.2001 - val_loss: 1.0997
Epoch 15/100
1/1 [==============================] - 0s 42ms/step - loss: 0.1891 - val_loss: 1.0944
Epoch 16/100
1/1 [==============================] - 0s 43ms/step - loss: 0.1779 - val_loss: 1.0893
Epoch 17/100
1/1 [==============================] - 0s 45ms/step - loss: 0.1667 - val_loss: 1.0845
Epoch 18/100
1/1 [==============================] - 0s 41ms/step - loss: 0.1555 - val_loss: 1.0798
Epoch 19/100
1/1 [==============================] - 0s 44ms/step - loss: 0.1444 - val_loss: 1.0754
Epoch 20/100
1/1 [==============================] - 0s 40ms/step - loss: 0.1334 - val_loss: 1.0712
Epoch 21/100
1/1 [==============================] - 0s 39ms/step - loss: 0.1227 - val_loss: 1.0673
Epoch 22/100
1/1 [=====

1/1 [==============================] - 0s 43ms/step - loss: 4.8914e-04 - val_loss: 1.0128
Epoch 95/100
1/1 [==============================] - 0s 46ms/step - loss: 4.6564e-04 - val_loss: 1.0128
Epoch 96/100
1/1 [==============================] - 0s 46ms/step - loss: 4.4330e-04 - val_loss: 1.0129
Epoch 97/100
1/1 [==============================] - 0s 47ms/step - loss: 4.2223e-04 - val_loss: 1.0129
Epoch 98/100
1/1 [==============================] - 0s 45ms/step - loss: 4.0249e-04 - val_loss: 1.0129
Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 1.0272 - val_loss: 0.3333
Epoch 2/100
1/1 [==============================] - 0s 50ms/step - loss: 1.0064 - val_loss: 0.3327
Epoch 3/100
1/1 [==============================] - 0s 43ms/step - loss: 0.9866 - val_loss: 0.3323
Epoch 4/100
1/1 [==============================] - 0s 42ms/step - loss: 0.9675 - val_loss: 0.3320
Epoch 5/100
1/1 [==============================] - 0s 39ms/step - loss: 0.9488 - val_loss: 0.3318
Epoch 6/10

Epoch 6/100
1/1 [==============================] - 0s 49ms/step - loss: 0.7254 - val_loss: 2.1484
Epoch 7/100
1/1 [==============================] - 0s 47ms/step - loss: 0.7076 - val_loss: 2.1372
Epoch 8/100
1/1 [==============================] - 0s 46ms/step - loss: 0.6900 - val_loss: 2.1257
Epoch 9/100
1/1 [==============================] - 0s 46ms/step - loss: 0.6723 - val_loss: 2.1140
Epoch 10/100
1/1 [==============================] - 0s 47ms/step - loss: 0.6543 - val_loss: 2.1021
Epoch 11/100
1/1 [==============================] - 0s 50ms/step - loss: 0.6359 - val_loss: 2.0902
Epoch 12/100
1/1 [==============================] - 0s 48ms/step - loss: 0.6169 - val_loss: 2.0781
Epoch 13/100
1/1 [==============================] - 0s 49ms/step - loss: 0.5973 - val_loss: 2.0659
Epoch 14/100
1/1 [==============================] - 0s 47ms/step - loss: 0.5770 - val_loss: 2.0536
Epoch 15/100
1/1 [==============================] - 0s 49ms/step - loss: 0.5560 - val_loss: 2.0411
Epoch 16/100
1

KeyboardInterrupt: 